In [6]:
#공통 import
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split, cross_validate

from joblib import parallel_backend
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

#모델 import
from xgboost import XGBClassifier, XGBRFClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

# df = pd.read_csv('./통합데이터/df_train_v3_mean.csv') # 결측치로 평균으로 한 테이블
df = pd.read_csv('./통합데이터/df_train_v3.csv') # 결측치를 제거한 테이블
df_test = df
df_test

# 컬럼명에서 'kma_lightbecon_train.'을 제거
df_test.columns = df_test.columns.str.replace('khnp_buoy_test.', 'k')
df_test.columns = df_test.columns.str.replace('kma_lightbecon_test.', 'l')
df_test.columns = df_test.columns.str.replace('kma_pagobuoy_test.', 'n')
df_train = df_test

# 데이터 전처리
data = df_train.drop(columns=['answer', 'year', 'month', 'day', 'hour'])
target = df_train['answer']
scaler = StandardScaler() #<- 스케일링 방식 변경 가능
data_scaled = scaler.fit_transform(data)

# PCA를 사용하여 차원 축소
pca = PCA(n_components = 10)  # 목표 차원 수로 조정
data_pca = pca.fit_transform(data_scaled)

# 축소된 데이터로 새로운 데이터프레임 생성
df_pca = pd.DataFrame(data_pca, columns=['PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6', 'PC7', 'PC8', 'PC9', 'PC10'])

# 축소된 데이터로 모델 훈련 데이터 나누기
X_train, X_val, y_train, y_val = train_test_split(df_pca, target, test_size=0.2, random_state=123, stratify=target)

# 모델 객체 생성
xgbc = XGBClassifier()
lgbm = LGBMClassifier(random_state=42)
rfc = RandomForestClassifier(n_estimators=100, max_depth=10, n_jobs=-1, random_state=42)
etc = ExtraTreesClassifier(random_state=42)
adaboost = AdaBoostClassifier(random_state=42)
xgbrfc = XGBRFClassifier()

# VotingClassifier로 앙상블 모델 생성
ensemble = VotingClassifier(estimators=[('rfc', rfc), ('xgbrfc', xgbrfc)], voting='hard') #<-모델명 변경에 따라 적절하게 변경

# 앙상블 모델 훈련
ensemble.fit(X_train, y_train)

# 훈련 세트 정확도 출력
train_pred = ensemble.predict(X_train)
train_accuracy = accuracy_score(y_train, train_pred)
print("훈련 세트 정확도:", train_accuracy)

# 검증 세트 정확도 출력
val_pred = ensemble.predict(X_val)
val_accuracy = accuracy_score(y_val, val_pred)
print("검증 세트 정확도:", val_accuracy)

C:\Users\sjo05\AppData\Local\Temp\ipykernel_5824\3109123500.py:27: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test.columns = df_test.columns.str.replace('khnp_buoy_test.', 'k')
C:\Users\sjo05\AppData\Local\Temp\ipykernel_5824\3109123500.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test.columns = df_test.columns.str.replace('kma_lightbecon_test.', 'l')
C:\Users\sjo05\AppData\Local\Temp\ipykernel_5824\3109123500.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  df_test.columns = df_test.columns.str.replace('kma_pagobuoy_test.', 'n')


훈련 세트 정확도: 0.9492860659773511
검증 세트 정확도: 0.8976377952755905
